In [6]:
import numpy as np
import random
from collections import Counter
from datetime import datetime
import keras
import sys
import io
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.optimizers import RMSprop
%reload_ext tensorboard

In [7]:
with open('data/Mushrooms.txt', 'r') as mushies:
    mushroom_accounts = mushies.read()

with open('data/Cannabis.txt', 'r') as pot:
    cannabis_accounts = pot.read()
    
with open('data/MDMA.txt', 'r') as molly:
    mdma_accounts = molly.read()
    
with open('data/LSD.txt', 'r') as acid:
    lsd_accounts = acid.read()

In [8]:
#returns a model trained on whatever text it is given. 
def make_model(text):
    chars = sorted(list(set(text)))
    char_indices = dict((c, i) for i, c in enumerate(chars))
    indices_char = dict((i, c) for i, c in enumerate(chars))

    seqlen = 40
    step = seqlen
    sentences = []
    for i in range(0, len(text) - seqlen - 1, step):
        sentences.append(text[i: i + seqlen + 1])

    x = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
    y = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)

    for i, sentence in enumerate(sentences):
        for t, (char_in, char_out) in enumerate(zip(sentence[:-1], sentence[1:])):
            x[i, t, char_indices[char_in]] = 1
            y[i, t, char_indices[char_out]] = 1


    logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

    model = Sequential()
    model.add(LSTM(128, input_shape=(seqlen, len(chars)), return_sequences=True))
    model.add(Dense(len(chars), activation='softmax'))


    model.compile(
        loss='categorical_crossentropy',
        optimizer=RMSprop(learning_rate=0.1),
        metrics=['categorical_crossentropy', 'accuracy']
    )


    def sample(preds, temperature=1.0):
        """Helper function to sample an index from a probability array."""
        preds = np.asarray(preds).astype('float64')
        preds = np.exp(np.log(preds) / temperature)  # softmax
        preds = preds / np.sum(preds)                #
        probas = np.random.multinomial(1, preds, 1)  # sample index
        return np.argmax(probas)                     #


    def on_epoch_end(epoch, _):
        """Function invoked at end of each epoch. Prints generated text."""
        print()
        print('----- Generating text after Epoch: %d' % epoch)

        start_index = random.randint(0, len(text) - seqlen - 1)

        diversity = 0.5
        
        print('----- diversity:', diversity)
        generated = ''
        sentence = text[start_index: start_index + seqlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, seqlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1
            preds = model.predict(x_pred, verbose=0)
            next_index = sample(preds[0, -1], diversity)
            next_char = indices_char[next_index]
            sentence = sentence[1:] + next_char
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

    print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

    model.fit(x, y,
              batch_size=128,
              epochs=14,
              callbacks=[print_callback])

    %tensorboard --logdir logs/scalars

In [9]:
mushrooms = make_model(mushroom_accounts)

Epoch 1/14
124905/124905 [==============================] - 169s 1ms/step - loss: 3.0765 - categorical_crossentropy: 3.0765 - accuracy: 0.3696

----- Generating text after Epoch: 0
----- diversity: 0.5
----- Generating with seed: "lt as foreign but the heart rate was an "
lt as foreign but the heart rate was an with them the styth or me that my completely partached to thing that I left to and the and the mushrooms (39) : Small hand that this of the rettle that I couldn't have to work and like the move of the tripping thing on the tripping the thought the trips was out the con intenter. I was the house the back on the wornverent and which and the an was since that I was a constanter later on the minutes i
Epoch 2/14
124905/124905 [==============================] - 147s 1ms/step - loss: 1.5204 - categorical_crossentropy: 1.5204 - accuracy: 0.5516

----- Generating text after Epoch: 1
----- diversity: 0.5
----- Generating with seed: "o my pockets. I looked at the landscape "
o my pockets.

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:40: RuntimeWarning: divide by zero encountered in log


an strange and started mention with my head my friends for a next of the chest are the time the time of my friends and sat that I had been a people in any hard the water I didn't would get the festions and the words and the store's distortions and was the experience with the first
Epoch 3/14
124905/124905 [==============================] - 144s 1ms/step - loss: 1.4863 - categorical_crossentropy: 1.4863 - accuracy: 0.5602

----- Generating text after Epoch: 2
----- diversity: 0.5
----- Generating with seed: "s I noticed this, beautifully brilliant "
s I noticed this, beautifully brilliant hard do before I really die dried my mind me and the depressimate even try to the part of the most trip with her than we then I had story period and the really strangers and experience the surrounding ming my open the world propering up that I had be with my friend. The end the ement that many trip experience the trip we had to started cheer and a really all concohard of the most more started to b
Epoc

124905/124905 [==============================] - 143s 1ms/step - loss: 1.5015 - categorical_crossentropy: 1.5015 - accuracy: 0.5608

----- Generating text after Epoch: 13
----- diversity: 0.5
----- Generating with seed: " fungus that we had been waiting on for "
 fungus that we had been waiting on for the experience: Not Given 
Published: Jul Experiences of a new I started to compaince in like I was already was to my head of looked in the time was by perfect in a love and order to do a set and over a shower my life to me on a bed.  I was a few trip and I was there is had with being a beautiful the song how to cart. I was there was a down them of I had a walk the each other and and the light and
